In [ ]:
%reload_ext dotenv
%dotenv

In [ ]:
import replicate
import requests
from PIL import Image

In [ ]:
garments = {
    "tshirt": {
        "description": "purple t-shirt",
        "category": "upper_body",
        "image_path": "../images/garments/tops/tshirt.webp",
    },
    "sweater": {
        "description": "oversized pink sweater",
        "category": "upper_body",
        "image_path": "../images/garments/tops/sweater.jpg",
    },
    "striped_sweater": {
        "description": "black and white striped sweater",
        "category": "upper_body",
        "image_path": "../images/garments/tops/striped_sweater.webp",
    },
    "winter_coat": {
        "description": "winter coat with fur lined hood",
        "category": "upper_body",
        "image_path": "../images/garments/tops/winter_coat.webp",
    },
    "raincoat": {
        "description": "light blue hip-length raincoat",
        "category": "upper_body",
        "image_path": "../images/garments/tops/raincoat.webp",
    },
    "jeans": {
        "description": "slim fit washed jeans",
        "category": "lower_body",
        "image_path": "../images/garments/bottoms/jeans.webp",
    },
    "joggers": {
        "description": "pink joggers",
        "category": "lower_body",
        "image_path": "../images/garments/bottoms/joggers.jpg",
    },
    "gym_shorts": {
        "description": "short white gym shorts",
        "category": "lower_body",
        "image_path": "../images/garments/bottoms/gym_shorts.webp",
    }
}

### Figure out where the garment might be

In [ ]:
def generate_mask(category):
    input = {
        # Garment doesn't matter if mask_only is True
        "garm_img": open(next(iter(garments.values()))["image_path"], "rb"),
        "human_img": open("../images/humans/model.jpg", "rb"),
        "category": category,
        "mask_only": True,
    }

    result_url = replicate.run(
        "cuuupid/idm-vton:c871bb9b046607b680449ecbae55fd8c6d945e0a1948644bf2361b3d021d3ff4",
        input=input,
    )
    
    return Image.open(requests.get(result_url, stream=True).raw)

In [ ]:
im = generate_mask("upper_body")
im.save("../images/masks/top.jpg")
im

### Put single garment on human

In [ ]:
def put_garment(garment):
    input = {
        "garm_img": open(garment["image_path"], "rb"),
        "human_img": open("../images/humans/model.jpg", "rb"),
        "mask_img": open(
            "../images/masks/top.jpg"
            if garment["category"] == "upper_body"
            else "../images/masks/bottom.jpg",
            "rb",
        ),
        "garment_des": garment["description"],
        "category": garment["category"],
    }

    result_url = replicate.run(
        "cuuupid/idm-vton:c871bb9b046607b680449ecbae55fd8c6d945e0a1948644bf2361b3d021d3ff4",
        input=input,
    )

    return Image.open(requests.get(result_url, stream=True).raw)

In [ ]:
im = put_garment(garments["striped_sweater"])
im.save("../images/results/single/striped_sweater.jpg")
im

### Combine garments

In [ ]:
from combining import combine_garments

human = Image.open('../images/humans/model.jpg')
print(human.format, human.size, human.mode)

mask_bottom = Image.open('../images/masks/bottom.jpg')
print(mask_bottom.format, mask_bottom.size, mask_bottom.mode)

mask_top = Image.open('../images/masks/top.jpg')
print(mask_top.format, mask_top.size, mask_top.mode)

result_bottom = Image.open('../images/results/single/jeans.jpg')
print(result_bottom.format, result_bottom.size, result_bottom.mode)

result_top = Image.open('../images/results/single/striped_sweater.jpg')
print(result_top.format, result_top.size, result_top.mode)

im = combine_garments(human, result_top, result_bottom, mask_top, mask_bottom)
# im.save("../images/results/multi/striped_sweater_jeans.jpg")
im